In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import random
import shutil
import math

import warnings
warnings.filterwarnings("ignore", module="matplotlib")

IMG_SIZE = 128
IMG_PER_FILE = 24000
BIN_PATH = './bin'

def conv_square(filename, size=IMG_SIZE, border=False):
    # img = cv2.imread(filename)
    try:
        img = plt.imread(filename)
    except:
        img = None

    # 이상한 이미지는 검은색으로 처리
    if img is None:
        img = np.zeros((IMG_SIZE, IMG_SIZE, 3), dtype=np.uint8)

    # 흑백 이미지 처리
    if len(img.shape) == 2:
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)

    # 알파체널 제거
    if img.shape[2] == 4:
        img = cv2.cvtColor(img, cv2.COLOR_RGBA2RGB)

    if border:
        h, w = img.shape[:2]
        ratio = w / h

        if ratio < 1:
            rimg = cv2.resize(img, (int(size * ratio), size))
        else:
            rimg = cv2.resize(img, (size, int(size / ratio)))

        # 정사각형으로 Padding

        rh, rw = rimg.shape[:2]
        bottom = int((size - rh) / 2)
        right = int((size - rw) / 2)
        rimg = cv2.copyMakeBorder(rimg, size - rh - bottom, bottom, size - rw - right, right, cv2.BORDER_CONSTANT)
    else:
        rimg = cv2.resize(img, (IMG_SIZE, IMG_SIZE))

    # 색상 변환하여 return
    return rimg  # cv2.cvtColor(rimg, cv2.COLOR_BGR2RGB)

def example_show(img_path):
    # 이미지 하나를 예시로 변환 후 출력
    rimg = conv_square(img_path)
    plt.imshow(rimg)
    plt.show()

In [3]:
if __name__ == '__main__':
    # 파일 이름 List 생성
    dog_path_list = [[1, a] for a in glob.glob('./data_train/Dog/*.jpg')]
    cat_path_list = [[0, a] for a in glob.glob('./data_train/Cat/*.jpg')]
    img_path_list = dog_path_list + cat_path_list
    random.shuffle(img_path_list)

    print(len(img_path_list))

    # 결과 출력할 디렉터리 삭제 & 생성
    #if os.path.exists(BIN_PATH):
    #    shutil.rmtree(BIN_PATH)
    #os.mkdir(BIN_PATH)

    # 결과 출력
    out_files_num = int(math.ceil(len(img_path_list) / IMG_PER_FILE))
    for num in range(out_files_num):
        if num == out_files_num - 1:
            filepath = os.path.join(BIN_PATH, 'test.bin')
        else:
            filepath = os.path.join(BIN_PATH, '%02d.bin' % num)
        with open(filepath, 'wb') as f:
            print(filepath)
            count = 0

            for _idx in range(IMG_PER_FILE):
                if (count % 1000 == 0):
                    print('%d' % count, end=' ')
                count += 1

                idx = _idx + IMG_PER_FILE * num
                if idx == len(img_path_list):
                    break
                item = img_path_list[idx]
                key = item[0]
                rimg = conv_square(item[1])

                label = np.array(item[0]).astype(np.uint8)
                data = np.array(rimg).astype(np.uint8)

                f.write(label.tobytes())
                f.write(data.tobytes())

            print()

    print('Done.')

25000
./bin\00.bin
0 1000 2000 3000 4000 5000 6000 7000 8000 9000 10000 11000 12000 13000 14000 15000 16000 17000 18000 19000 20000 21000 22000 23000 
./bin\test.bin
0 1000 
Done.
